https://aws.amazon.com/blogs/machine-learning/bring-your-own-pre-trained-mxnet-or-tensorflow-models-into-amazon-sagemaker/

# Prerequisites

* After training a custom estimator, the model is saved in an exportable format using `tf.estimator.Estimator.export_saved_model()` method (see "Tensorflow Estimator/Custom Estimator.ipynb" for more details).
* Move the TensorFlow exported model into a directory export\Servo. Amazon SageMaker will recognize this as a loadable TensorFlow model. Your directory and file structure should look like this:
![SegmentLocal](resources/pre-trained-models-sagemaker-2.gif)
* SavedModel directory structure:
![SegmentLocal](resources/1.png)
* Model inputs and outputs:

`The given SavedModel SignatureDef contains the following input(s):
  inputs['PetalLength'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder_2:0
  inputs['PetalWidth'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder_3:0
  inputs['SepalLength'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder:0
  inputs['SepalWidth'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: Placeholder_1:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['classes'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: ArgMax:0
  outputs['probabilities'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 3)
      name: Softmax:0
Method name is: tensorflow/serving/predict`

# Overview

The following is an overview of the entire process.

* Step 1: Model definitions are written in a framework of choice.
* Step 2: The model is trained in that framework.
* Step 3: The model is exported and model artifacts that can be understood by Amazon SageMaker are created.
* Step 4: Model artifacts are uploaded to an Amazon S3 bucket.
* Step 5: Using the model definitions, artifacts, and the Amazon SageMaker Python SDK, a SageMaker model is created.
* Step 6: The SageMaker model is deployed as an endpoint.

![SegmentLocal](resources/pre-trained-models-sagemaker-1.gif)

# Upload model artifacts to Amazon S3

Tar the entire model directory (`export` directory) and upload to Amazon S3

* Tar the model directory

```python
import os
import tarfile

export_model_dir = os.path.join('model','export')
model_artifact_path = os.path.join('model', 'aws', 'model.tar.gz')
with tarfile.open(model_artifact_path, mode='w:gz') as archive:
    archive.add(export_model_dir, recursive=True)
```

* Create an Amazon S3 Bucket (skip this step if you had already created an S3 bucket)

  https://docs.aws.amazon.com/sagemaker/latest/dg/gs-config-permissions.html
  
  Go to aws console (https://console.aws.amazon.com). Select <b>Services</b>. Select <b>Storage/S3</b> and create a bucket
  
  Note: Include "sagemaker" in the bucket name. For example, "sagemaker-<i>[datetime]</i>".
  
* Upload model artifacts (`model.tar.gz`) to the S3 bucket.

  Model path example:
  
  `s3://sagemaker-22072019/tensorflow-custom-estimator/iris/model/model.tar.gz`

# Deploy from model artifacts

* Create an Amazon SageMaker Notebook Instance

  https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html

  1. Open the Amazon SageMaker console at https://console.aws.amazon.com/sagemaker/.
  2. Choose <b>Notebook instances</b>, then choose <b>Create notebook instance</b>.
  3. On the <b>Create notebook instance</b> page, provide the following information (if a field is not mentioned, leave the default values):
    * For <b>Notebook instance name</b>, type a name for your notebook instance. E.g. `Deploy-pretrained-iris-custom-estimator`
    * For Instance type, choose `ml.t2.medium`.
    * For IAM role, choose Create a new role, then choose Create role. The IAM role you create will have permission to access to any S3 bucket with "sagemaker" in the name
  4. Launch and create a new notebook

* Deploying directly from model artifacts

```python
from sagemaker import get_execution_role
from sagemaker.tensorflow.serving import Model

role = get_execution_role()
model = Model(model_data='s3://sagemaker-22072019/tensorflow-custom-estimator/iris/model/model.tar.gz', 
              role=role, framework_version='1.13')
              
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
```

# Making predictions against a SageMaker Endpoint

```python
inputs = {
    "instances": 
    [{"SepalLength": 6.4, "SepalWidth": 2.8, "PetalLength": 5.6, "PetalWidth": 2.2},
     {"SepalLength": 4.9, "SepalWidth": 3.1, "PetalLength": 1.5, "PetalWidth": 0.1}]
}
result = predictor.predict(inputs)
print(result)
```

`{
    'predictions': [{'probabilities': [1.02038e-09, 0.000515907, 0.999484], 'classes': 2},
                    {'probabilities': [0.999027, 0.000972887, 1.89823e-09], 'classes': 0}]
}`

Use Postman to test the api
* Get your access key and secret key for authorization
  * Open the Amazon IAM console at https://console.aws.amazon.com/iam/.
  * Choose <b>Users</b> and select your user
  * Choose <b>Security credentials</b>, then choose <b>Create access key</b>
  * Get your access key and secret key.
* Get the endpoint url
  * Open the Amazon SageMaker console at https://console.aws.amazon.com/sagemaker/.
  * Choose <b>Endpoints</b>, then choose your deployed model endpoint.
  * Get the endpoint url
* Launch Postman:
  * For <b>Authorization</b>, choose <b>AWS Signature</b>
    * Enter your AccessKey and SecretKey.
    * For AWS Region, enter your region.
    * For Service Name, enter "sagemaker"
  * For url, enter the endpoint url. Use the POST method.
  * For message body, enter
```json
{
    "instances": [
        {
            "SepalLength": 6.4,
            "SepalWidth": 2.8,
            "PetalLength": 5.6,
            "PetalWidth": 2.2
        },
        {
            "SepalLength": 4.9,
            "SepalWidth": 3.1,
            "PetalLength": 1.5,
            "PetalWidth": 0.1
        }
    ]
}
```

Result:

```json
{
    "predictions": [
        {
            "probabilities": [
                1.02038e-9,
                0.000515907,
                0.999484
            ],
            "classes": 2
        },
        {
            "probabilities": [
                0.999027,
                0.000972887,
                1.89823e-9
            ],
            "classes": 0
        }
    ]
}
```